<a href="https://colab.research.google.com/github/korjavin/random-notebooks/blob/master/LSD_stylegan3_v1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# В этом колабе ты можешь сгенерировать музыкальное видео с помощью алгоритмов **StyleGAN3** и **Lucid Sonic Dreams**
---
### чтобы все заработало, надо по очереди выполнить все ячейки нажав на кнопки **play** слева от них. только помни: пока выполняется одна ячейка, другая запуститься не может.
если ты в колабе первый раз, стоит убедиться что вычисления будут происходить на видеокарте. заходи в `Runtime >> Change runtime type >> GPU` (если все на русском то `Среда выполнения >> Сменить среду выполнения >> GPU`)

### если по какой то причине ты видишь код, попробуй справа над ячейкой нажать на кнопку с тремя точками и выбрать `hide code` или `спрятать код`


[StyleGAN3](https://github.com/NVlabs/stylegan3),
[оригинальный Lucid Sonic Dreams](https://github.com/mikaelalafriz/lucid-sonic-dreams),
[Lucid Sonic Dreams - Pytorch](https://github.com/NotNANtoN/lucid-sonic-dreams)

и не забудь подписаться на канал [тоже моушн](https://t.me/too_motion)

In [2]:
#@title установка Lucid Sonid Dreams и StyleGAN3
from IPython.display import clear_output
%cd /content/
!git clone https://github.com/dvschultz/lucid-sonic-dreams
!pip install pygit2 mega.py ninja
%cd /content/lucid-sonic-dreams
!git clone https://github.com/NVlabs/stylegan3.git
!mv /content/lucid-sonic-dreams/stylegan3 /content/lucid-sonic-dreams/stylegan2
clear_output()

In [ ]:
#@title скачивание предобученных моделей. если в названии есть `r` или `t`, это модели StyleGAN3 и картинка в кадре будет меняться по разному, либо слегка поворачиваться либо заметно перемещаться
from IPython.display import clear_output

model_to_download = "animals-r" #@param ["snowflakes", "wikiart-t", "animals-t", "animals-r", "faces-t", "faces-r","artistic_faces-t","artistic_faces-r","cats","dogs","wild","minerals","acid_fluid","earth_view"]
if model_to_download == "wikiart-t":
  !gdown --id 18MOpwTMJsl_Z17q-wQVnaRLCUFZYSNkj # wikiart
elif model_to_download == "snowflakes":
  !gdown --id 12VK87A3AXsVksdwcsJBUOETyUyCdP0nX # snowflakes
elif model_to_download == "animals-t":
  !wget https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-t-afhqv2-512x512.pkl # animals-t
elif model_to_download == "animals-r":
  !wget https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-afhqv2-512x512.pkl # animals-r
elif model_to_download == "faces-t":
  !wget https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-t-ffhq-1024x1024.pkl # faces-t
elif model_to_download == "faces-r":
  !wget https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-ffhq-1024x1024.pkl # faces-r
elif model_to_download == "artistic_faces-t":
  !wget https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-t-metfaces-1024x1024.pkl # artistic faces-t
elif model_to_download == "artistic_faces-r":
  !wget https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-metfaces-1024x1024.pkl # artistic faces-r
#elif model_to_download == "cosplay_faces":
#  !wget https://l4rz.net/cosplayface-snapshot-004000-18160-FID367.pkl # cosplay_faces
elif model_to_download == "cats":
  !wget https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan2/versions/1/files/stylegan2-afhqcat-512x512.pkl # cats
elif model_to_download == "dogs":
  !wget https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan2/versions/1/files/stylegan2-afhqdog-512x512.pkl # dogs
elif model_to_download == "wild":
  !wget https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan2/versions/1/files/stylegan2-afhqwild-512x512.pkl # wild
elif model_to_download == "minerals":
  !gdown --id 1YKkozuo2Ik_SGMwh-mFH0I9ZzCN2_gTn # minerals
elif model_to_download == "acid_fluid":
  !gdown --id 10IPazsui4eBPAiLi1p8iMSqL9h0hcqH0 # acid fluid
elif model_to_download == "earth_view":
  !gdown --id 15ypXiZatt2zYvA805vKoMbjH-iU-2IFD # earth view

clear_output()

In [5]:
!cp -r /content/drive/MyDrive/projected_gan/pg_modules ./
!pwd
!pip install timm

/content/lucid-sonic-dreams
     |████████████████████████████████| 431 kB 5.1 MB/s 


In [10]:
from pg_modules.discriminator import ProjectedDiscriminator
D = ProjectedDiscriminator()

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/tf_efficientnet_lite0-0aa007d2.pth" to /root/.cache/torch/hub/checkpoints/tf_efficientnet_lite0-0aa007d2.pth


In [13]:
D.feature_network.requires_grad_(False)

F_RandomProj(
  (pretrained): Module(
    (layer0): Sequential(
      (0): Conv2dSame(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
      (3): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act1): ReLU6(inplace=True)
          (se): Identity()
          (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act2): Identity()
        )
      )
      (4): Sequential(
        (0): InvertedResidual(
          (conv_pw): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(96, eps=0.001

In [6]:
#@title прежде чем запускать эту ячейку, надо открыть слева панель файлов (нажать на иконку папки) и загрузить туда аудио файл. когда файл загрузится, надо нажать по нему правой кнопкой мыши, скопировать путь и вставить в строку ниже

from lucidsonicdreams import LucidSonicDream

music = '/content/drive/MyDrive/Retro-short.mp3' #@param {type:"string"}

#@markdown ###затем в папке `lucid-sonid-dreams` нужно найти скачанную модель с расширением `.pkl` и так же скопировать ее путь в строку ниже
model = '/content/drive/MyDrive/projected_gan/training-runs/00001-stylegan2-art_elks1024-gpus1-batch64-/network-snapshot.pkl' #@param {type:"string"}
#@markdown ###а в следующей строке надо указать название будущего файла
#@markdown ###*ВАЖНО: файл перезапишется если ему не поменять название при повторном запуске ячейки*
final_video_name = 'LSD_animal.mp4' #@param {type:"string"}

#@markdown ###следующие настройки для теста можно оставить как есть. подробный разбор всех параметров на английском можно найти в [оригинальном колабе](https://colab.research.google.com/drive/1Y5i50xSFIuN3V4Md8TB30_GOAtts7RQD#scrollTo=GuLiMg9CUXvG)

L = LucidSonicDream(song = music, #mp3 here
                    style = model) #pkl path here

L.hallucinate(file_name = '/content/'+ final_video_name, #output path here
              start = 0 #@param {type:"integer"}
              ,
              duration=50 #@param {type:"integer"}
              ,
              resolution = 1026 #@param {type:"integer"}
              ,  
              speed_fpm = 11 #@param {type:"integer"}
              ,
              fps = 30 #@param {type:"integer"}
              ,
              motion_react = 0.8 #@param {type: "slider", min: 0.0, max: 2.0, step: 0.05}
              ,
              motion_randomness = 0.5 #@param {type: "slider", min: 0.0, max: 1.0, step: 0.05}
              ,
              pulse_react = 1 #@param {type: "slider", min: 0.0, max: 2.0, step: 0.05}
              ,
              random_seed=6637 #@param {type:"slider", min:0, max:9999, step:1}
              ,
              ) 

Preparing style...
Loading networks from /content/drive/MyDrive/projected_gan/training-runs/00001-stylegan2-art_elks1024-gpus1-batch64-/network-snapshot.pkl...
Preparing audio...
Loading effects...
Num classes of model:  0


Doing math...



Hallucinating... 



Generating frames:   0%|          | 0/1567 [00:00<?, ?it/s]


AssertionError: ignored

In [ ]:
!cp /content/LSD_animal.mp4 /content/drive/MyDrive/LSD_me2.mp4

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##а если умеешь работать с кодом, все то же самое можно сделать в следующей ячейке

In [ ]:
from lucidsonicdreams import LucidSonicDream

L = LucidSonicDream(song = '/content/Leikeli47 - Money.mp3', #mp3 here
                    style = '/content/lucid-sonic-dreams/stylegan3-t-afhqv2-512x512.pkl') #pkl path here

L.hallucinate(file_name = '/content/basic-lsd.mp4', #output path here
              start = 0,
              duration=38,
              resolution = 640,  
              speed_fpm = 8,
              fps = 30,
              pulse_percussive = True,
              pulse_harmonic= False,
              pulse_react = 1.0,
              motion_react = 0.8,
              motion_randomness = 0.8,
              motion_harmonic = True,
              motion_percussive = False,
              random_seed=100,
              ) 